In [2]:
import pickle
import os
import tweepy
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
import csv
import pandas as pd
import numpy as np
import json

In [6]:
if not os.path.exists('secret_twitter_credentials_8_jun_21.pkl'):
    Twitter={}
    Twitter['Consumer Key'] = ''
    Twitter['Consumer Secret'] = ''
    with open('secret_twitter_credentials_8_jun_21.pkl','wb') as f:
        pickle.dump(Twitter, f)
else:
    Twitter=pickle.load(open('secret_twitter_credentials_8_jun_21.pkl','rb'))

In [8]:
auth = tweepy.AppAuthHandler(Twitter['Consumer Key'], Twitter['Consumer Secret'])

In [9]:
api = tweepy.API(auth)
for tweet in tweepy.Cursor(api.search, q='cardinal health').items(1):
    #print(tweet.user.screen_name)
    #print(tweet.possibly_sensitive)
    #print(tweet._json.keys())
    print(tweet._json['user'].keys())
    #print(tweet._json['user']['followers_count'])

dict_keys(['id', 'id_str', 'name', 'screen_name', 'location', 'description', 'url', 'entities', 'protected', 'followers_count', 'friends_count', 'listed_count', 'created_at', 'favourites_count', 'utc_offset', 'time_zone', 'geo_enabled', 'verified', 'statuses_count', 'lang', 'contributors_enabled', 'is_translator', 'is_translation_enabled', 'profile_background_color', 'profile_background_image_url', 'profile_background_image_url_https', 'profile_background_tile', 'profile_image_url', 'profile_image_url_https', 'profile_banner_url', 'profile_link_color', 'profile_sidebar_border_color', 'profile_sidebar_fill_color', 'profile_text_color', 'profile_use_background_image', 'has_extended_profile', 'default_profile', 'default_profile_image', 'following', 'follow_request_sent', 'notifications', 'translator_type', 'withheld_in_countries'])


In [10]:
def scrape_by_search(search_term,num_results):
    api = tweepy.API(auth)
    results = pd.DataFrame()
    for i,tweet in enumerate(tweepy.Cursor(api.search, q=search_term).items(num_results)):
        for t in list(tweet._json.keys()):
            try:
                results.loc[i,t] = tweet._json[t]
            except:
                pass
        for user_t in list(tweet.user._json.keys()):
            if user_t == 'created_at':
                user_t = 'profile_created_at'
            try:
                results.loc[i,user_t] = tweet.user._json[user_t]
            except:
                pass
    results['created_at_date'] = list(map(lambda x: x.strftime('%Y-%m-%d'),pd.to_datetime(results.created_at)))
    results['created_at_hour'] = list(map(lambda x: x.strftime('%H'),pd.to_datetime(results.created_at)))
    return results
df = scrape_by_search('cardinal health',5)
df.head()

,created_at,id,id_str,text,truncated,entities,metadata,source,in_reply_to_status_id,in_reply_to_status_id_str,...,default_profile,default_profile_image,following,follow_request_sent,notifications,translator_type,withheld_in_countries,possibly_sensitive,created_at_date,created_at_hour
0,Fri Jul 16 00:26:23 +0000 2021,1.262464e+18,1262464138221142018,@DrAllisonBarber @AnthemInc While your husband...,False,description,NaN,"<a href=""http://twitter.com/download/android"" ...",1.415723e+18,1415723179016925185,...,True,False,NaN,NaN,NaN,none,NaN,NaN,2021-07-16,00
1,Thu Jul 15 20:29:20 +0000 2021,2.562251e+08,256225105,Filling a single compliance package by hand ca...,True,NaN,NaN,"<a href=""https://www.hootsuite.com"" rel=""nofol...",NaN,None,...,False,False,NaN,NaN,NaN,none,NaN,False,2021-07-15,20
2,Thu Jul 15 19:56:33 +0000 2021,9.634956e+17,963495628545327104,$CAH NEW ARTICLE : Cardinal Health launches Na...,True,NaN,NaN,"<a href=""https://www.stck.pro"" rel=""nofollow"">...",NaN,None,...,True,False,NaN,NaN,NaN,none,NaN,False,2021-07-15,19
3,Thu Jul 15 18:55:26 +0000 2021,5.789636e+08,578963608,Are you attending the 2021 Cardinal Health RBC...,True,NaN,NaN,"<a href=""http://www.hubspot.com/"" rel=""nofollo...",NaN,None,...,False,False,NaN,NaN,NaN,none,NaN,False,2021-07-15,18
4,Thu Jul 15 18:54:12 +0000 2021,2.317454e+09,2317453908,Are you attending the 2021 Cardinal Health RBC...,True,NaN,NaN,"<a href=""http://www.hubspot.com/"" rel=""nofollo...",NaN,None,...,False,False,NaN,NaN,NaN,none,NaN,False,2021-07-15,18


In [14]:
df.text[0]

'@DrAllisonBarber @AnthemInc While your husband is helping push unregulated opioid trafficking from Cardinal Health.'

In [15]:
import nltk
from nltk.corpus import twitter_samples
nltk.download("stopwords")

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidanc

True

In [17]:
twitter_samples.fileids()

['negative_tweets.json', 'positive_tweets.json', 'tweets.20150430-223406.json']

In [18]:
twit_samples_corpus = ([(t,'1')for t in twitter_samples.strings("positive_tweets.json")] + 
             [(t,'-1') for t in twitter_samples.strings("negative_tweets.json")])

In [57]:
pos_tweets = pd.DataFrame({'tweets':twitter_samples.strings("negative_tweets.json"),
              'sentiment':[1]*len(twitter_samples.strings("negative_tweets.json"))})
neg_tweets = pd.DataFrame({'tweets':twitter_samples.strings("negative_tweets.json"),
              'sentiment':[-1]*len(twitter_samples.strings("negative_tweets.json"))})
twit_samples_corpus = pd.concat([pos_tweets,neg_tweets])\
                        .sample(frac = 1)\
                        .reset_index(drop = True)
twit_samples_corpus.head()

,tweets,sentiment
0,no motivation :(,1
1,:( exit gomen,-1
2,"@jmcefalas Jeebus. Reabsorbtion of tooth, absc...",1
3,lost all the excitement for my birthday :-(,1
4,&amp; why tf can't i find subtitles for OITNB ...,1


In [20]:
stopwords = nltk.corpus.stopwords.words("english")

In [51]:
def build_a_bag_of_words_features(words):
    words = nltk.word_tokenize(words)
    return {
        word: True for word in words \
        if not word in stopwords}

In [52]:
build_a_bag_of_words_features(df.text[4])

{'Are': True,
 'attending': True,
 '2021': True,
 'Cardinal': True,
 'Health': True,
 'RBC': True,
 'virtual': True,
 'event': True,
 '?': True,
 'Stop': True,
 'booth': True,
 'enter': True,
 'raffle': True,
 '$': True,
 '100': True,
 'gif…': True,
 'https': True,
 ':': True,
 '//t.co/45FYy4LI8W': True}

In [53]:
def token_bag_words(data,label):
    i=0 
    length = len(data)
    tokenized_tweets = []
    while i<length:
        tokenized_tweets.append(build_a_bag_of_words_features(data[i]))
        i+=1
    #return tokenized_tweets
    j=0
    clean_tweets=list(np.arange(len(tokenized_tweets)))
    while j<len(tokenized_tweets):
        clean_tweets[j]=[tokenized_tweets[j],label]
        j+=1
    clean_tweets = tuple(clean_tweets)
    return clean_tweets

In [82]:
def token_bag_words(data,source):
    df = pd.DataFrame()
    for i, tweet in enumerate(data.tweets):
        print(tweet)
        df.loc[i,'tweet'] = tweet
        df.loc[i,'source'] = source
        df.loc[i,'sentiment'] = data.loc[i,'sentiment']
        for word,_ in build_a_bag_of_words_features(tweet).items():
            df.loc[i,word] = 1
    return df

In [86]:
twit_samples_corpus

,tweets,sentiment
0,no motivation :(,1
1,:( exit gomen,-1
2,"@jmcefalas Jeebus. Reabsorbtion of tooth, absc...",1
3,lost all the excitement for my birthday :-(,1
4,&amp; why tf can't i find subtitles for OITNB ...,1
...,...,...
9995,@LeanneHirst @3Yorkshireteers oh and our conve...,-1
9996,"Ok, back to work :( http://t.co/QFtLZ2Cl0w",-1
9997,I miss abudhabi :(,-1
9998,Just wanna go back to my dream 🎀 was buying lo...,1


In [85]:
test = token_bag_words(twit_samples_corpus,'corpus')

no motivation :(
:( exit gomen
@jmcefalas Jeebus. Reabsorbtion of tooth, abscess, threatening to affect my lovely front crown :( and OOOOOOOOUCH!!!
lost all the excitement for my birthday :-(
&amp; why tf can't i find subtitles for OITNB ? I need to understand Changs backstory :(
So no one about to come smoke with me? :(
.@uberuk you cancelled my ice cream uber order. Everyone else in the office got it but me. :(
@jungsilhoon i do like ballads but idk :((
Of course I always miss @selenagomez's Tweeting sprees :(
Ufffff Sr Dard :( — drinking green tea
@twyodor @gkjohn :-( I don't read most of these rehash websites that have mushroomed. A good piece may be by exception &amp; reaches one anyway.
@HoneyMustardHoe :( I was kidding, get out your feelings
Im crying right now my babies :(
@lmaoclffrd yeah sad life :((
BECAUSE THAT'S YOUR GOAT MINO : (
Practice sad lage :(
I'm really just up :(
@WingsScotland no cats :-(
@RabihAntoun :( so sad for us. We're losers
Alone :-( :'( :-\
whats wrong 

@JamesMorrisonOK gutted !! I couldn't get tickets and it was my birthday :( are you releasing more dates
@lifelesscurves they're really unique and refreshing :( the popularity was great too. Just why...just why &gt;_&gt;
@BacheloretteABC @kaitlynbristowe would love to see her w/Nick but if she picks no one I waisted TV time. &amp; they never work out anyway. :(
this time in 20 days i'll have my results kms :(
Shanzay salabraty :(((
@taesprout god im so sorry i hope you are better now :-( wah
Who even started this trend? I wanna know if there is a jot of truth :( #ZaynIsComingBackOnJuly26
So, its been 40 days since you been gone and I still missing you like crazy bcs there were many hard times that I need you to support me :(
@gfcakelover No, not yet :( .... but hopefully soon. Fingers crosssed! Are you coming to the @foodiesfestival in #clapham
@chaisooyoung dare :(
"@alyaeldeeb12345: I miss Chris ' voice :(" same http://t.co/5houV3sq5l
@scubadiver5 @Catharine34 I'm keeping the bug spr

@jonathan_wolle Lucky :(
@ImJoanneT Ahh no what did you get done?? I hope you're okay :(
I still want a cactus :( https://t.co/TD8A5vEc9p
We're not going on the cable cars now because my brother got scared and doesn't want to :(((
where can i get long lasting black pumps :(
@the_rebel_in_me @swanqueentacos awww. Imperative. :( #TRMDHesitant
I wish I had Cara Delevingne's face :-(
@selenagomez why did you have answer fans when i was sleep ??? :(
@_khooie :( I went all in too (wasn't much) I swear I'm the unluckiest bettor...
#FreebieFriday  fingers crossed had to cut down food shop cos unexpected garage bill this week :(  https://t.co/k7bFXN9H5V
Another dissapointed review :( http://t.co/HlnZzdW0Tz #bbloggers #makeup #beauty @BBlogRT @BBloggersUnite @FemaleBloggerRT
so after 2 weeks only mockingjay was left well that was slow :(
@charmsham i can't able to view pics da.. :( wat pic is this?
Mood: cumbia :((((😩
im still giddy over d1 :-(
@zedxgarcia u started it :(
@blockedmefat dont unst

@seunrgriseyo good dong by?:(
sini dipoppo😘😘😘😘
Whenever I would spend the night we would smoke and watch a movie but he would always end up falling asleep on me :(
I doubt i'm going to bed soon though. :( I wake up at like 9.
@Zerbrechend SAME :(
Headache :(
@RocketLeague you have a due date for the fix of error 59 ? My friend and me can't still play :(
someone has niall layouts to give me? :(
@wotjustin UR ACTUALLY WATCHING (but is it for me smh #what #a #bff #yes #hashtags #bc #u #love #me jk omfg im so hyper :-( )
Poor guy :( http://t.co/aSq9NN4AXR
.... My gosh ... I'm really bad at physics :(
@tanginnomo no he kept looking at my bag :(
@howarth_83 Hi Paul, that's not good to hear :( Can you DM some more information about what happened when you contacted us?
Thankyou sa treat kanina and sorry sa breakdown bc of Sir Mags :(( @ZAIRERR
@TheKelseeey awhhh ok ok :( see you nalang when class opens!!! Hehe
no more of being 'high all the time' :((((((((((( this is v sad
@cmonlukey all of th

It sucks so much been sick i was plan to start work on my first gundam to night but nope. :(
@RiotQuickshot Good man! Shame he no longer has a Cigar though :(
#PARTY goodbye stage :(
@1q4h_ cheer up!! :-( 😘💖
I know :( askies "@Titus3D: @Nessa_Mbeki hai that's what u said last time"
i wanna go to vidcon :(
i need nate ruess' album :(
@Taeyeonniex3 so they didn't win on their 2nd stage of mubank? :(
last summer vacation with my family im emotional :(
I feel lonely someone talk to me guys and girls :(

@TheOnlyRazzYT @imarieuda @EiroZPegasus @AMYSQUEE @UdotV
My KIK : oulive70748 #kik #kikmenow #photo #babe #loveofmylife #brasileirao #viernesderolenahot :( http://t.co/GmmwD4PrhU
Used to be so skinny :( http://t.co/PQyN3HNJ33
@dumplinghoya I've been wanting to change it to only woohyun one but sunggyu makes me feel guilty :(
I need to clear my head :(
@CNET Thats why they been pushing emoji, I give Sony five  :-(  :-(  :-(  :-( :-(  :-(
@iMac_too impossible sir for NM to change the order in

...

@JayMcGuiness  :-( please notice mew
@ASUS_ROGUK @OC3D That's what she said! .... :(
@llama_ajol eat and rest well :( you're too tired..
Edsa y u do dis 2 me

So tired. Just wanna go home :(
Missed 11:11 :( btw my wish was for @carterreynolds to follow me♥ #LoveYouTillTheEndCarter ☺
Wish I was going to Leeds festival. Need to see Simple Plan :( it's been too long!
Life never felt so good :(
Unfortunately today's performances at Pier Approach will not be going ahead due to the weather. :( http://t.co/PidhDm1ga3
@DeviousLiz awh :-( 💙 but you have to stay 💪
@kargadouri FOLLOWED ME THANKS, AND
@justinbieber PLEASE FOLLOWED ME TOO :(
@LuiCalibre I'd be down but man i have to work right now :(
@fawadchaudhry May ALLAH save us from Pakistan`s Juudiciary :(
Paper towns please :-(
I don't know how to start my requirements :(
Nobody said it was easy :(
Having boring time :( don't know what to do.....
sick of this feeling. :( i just wanna be happpy
I'm too tan :-(((
im so sure fany is so happy no

😂😂😂"@Danny_boym: Lol 👌"@sick_fruits: :( @_kydd_que: LMFAOO..So Accurate!!! http://t.co/k4daapzfUz""
please follow me :( @JackJackJohnson
Nakaka Beastmode. I lost my ID :(
:((((((( this is so sad i cwnt help it  https://t.co/glBmiAAJ79
@21oclock :((( bout to instant transmission
@subsubjjang YEAH HE DID BC AT THE END HE RUN BACK TO HIS PLACE HAHAHA I DIDNT SAW THEM LAUGHING THO BC LOW QUALITY :(
My body system don't want to accept those foods :(
ya i did :-( https://t.co/peQqNxkDj0
@JabongIndia very disappointed :( never ever been lucky from your handle. #JabongatPumaUrbanStampede
phone fell out of my hand and it woke me up :(
@_awkwardraven Will do Senpai &gt;:( -looks up on all of buttsex-
@eleena_pv @apuchades I'm sorry :(
Bale :(
♛♛♛
》》》》 
I LOVE YOU SO MUCH.
I BELİEVE THAT HE WİLL FOLLOW.
PLEASE FOLLOW ME PLEASE JUSTİN @justinbieber :( x15.338
》》》》ＳＥＥ ＭＥ
♛♛♛
@LittleMix @SonyMusicNL Can u make a diary while you're all there please :((
"@alyaeldeeb12345: I miss Chris ' voice :(" same

✈️2 days and I'm back in rainy UK :-(☔️
@Ni_All_Is_Bae aww I don't really wanna get my hopes up but I wish that he would :((
Nerve wracking test. Booset. :(
Short Talk! Last Day Of Beach :(: http://t.co/sStJosDrNZ via @YouTube
LOL this time i mean it :( @Katz360_ You and time 😊 https://t.co/KQ2pDd30w8 …
the only thing i miss about Ireland is the cheese :(
@Espio1 @scully1888 It's madness, for sure. Three amiibo for the price of four on Nintendo store. It's just my backup… :(
@Mr_FRML sir plus 4 please :((
@Thirty3forty5 Yeah - we thought that was part of the tour we did  today, but it wasn't. :-(
school starts pretty soon :(
@nuriaagonzalez it was amazing!! Such a short trip though :( I wanna go again, but for more days!! How's Spain? X
@StayChill69 I Let You Down And Now I Feel Like Complete Shit. :(
I was about to make tea but load shedding was like Ha.a bheyps not now ! Ayemso Hacked :(((( http://t.co/ZudfTknvCM
@Glam_And_Gore it's noon here where I live so I was like...ouh differen

@curlsharryxoxo i don't know :(( just be happy for him :(
@SynergyFlying No!!! Why did you delete me?:(
Llama, are u sick? :( pls take care of urself #GetWellSoonAmber @llama_ajol
rip whitney houston. :(
No give work again your old staff come police later my be court.  I look another job.  :(  bye bye.
@ollgbroxei sc, longe :(
Feeling a little neglected as there has been no visit from the postie today! :-(
@curiouskittten didn't even know you still existed on here haha text me :( 626-430-8715
@lilaclarents Oh no sorry to hear this :( please can you email the information to pro@illamasqua.com instead and they can manually set up? x
I can't finish my Sanum today! :(
#LLAOLLAO #Dessert but too full
I feel like I haven't been in Twitter in like 5ever :-(
Again not feeling well :(
@zeynepirdal FOLLOWED ME THANKS, AND
@justinbieber PLEASE FOLLOWED ME TOO :(
No words can explain how much i miss you. Hope you know that.  :( @zaynmalik
want chicken nuggets again :(
@jojosmith1964 Oh no :( sorry

KeyboardInterrupt: 

In [ ]:
test

In [67]:
twit_samples_corpus

,tweets,sentiment
0,no motivation :(,1
1,:( exit gomen,-1
2,"@jmcefalas Jeebus. Reabsorbtion of tooth, absc...",1
3,lost all the excitement for my birthday :-(,1
4,&amp; why tf can't i find subtitles for OITNB ...,1
...,...,...
9995,@LeanneHirst @3Yorkshireteers oh and our conve...,-1
9996,"Ok, back to work :( http://t.co/QFtLZ2Cl0w",-1
9997,I miss abudhabi :(,-1
9998,Just wanna go back to my dream 🎀 was buying lo...,1
